<a href="https://colab.research.google.com/github/Vladimirsp81/DLS_2/blob/master/HW_BERT_Vladimir_Prudnikov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework

Привет! В этой домашнем задании ты научишься обучении модели BERT. На семинаре был разобран код модели, здесь же посмотрим на то, как надо обработать данные, чтобы на них модель могла учиться. 

Замечания по выполнению задания:

- Код внутри блока `<DON'T TOUCH THIS!>` используется для проверки задания, его нельзя трогать. 

- Внутри блока `<YOUR CODE>` может больше кода, чем там показано изначально.

- От задания требуется написания небольшого отчета в конце.


Для начала загрузи нужные библиотеки.

In [446]:
!pip install transformers catalyst

In [0]:
import os
import random
import sys
import urllib.request
import zipfile

import numpy as np
import pandas as pd
from tqdm import tqdm, trange

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, RandomSampler, Dataset

import transformers

from catalyst.dl import SupervisedRunner
from catalyst.dl.callbacks import AccuracyCallback, SchedulerCallback, F1ScoreCallback
from catalyst.utils import set_global_seed, prepare_cudnn

Внизу идет технический код, который нужен для загрузки датасетов. Его можно уменьшить, выбрав только некоторые из них. Для того, что бы зачесть задание, надо выбрать не менее двух задач, для хотя бы одной из которых нужно использовать два предложения(ответ и вопрос, два предложения и прочее). Подробнее про датасеты [здесь](https://gluebenchmark.com/).

In [0]:
TASKS = ["CoLA", "SST", "MRPC", "QQP", "STS", "MNLI", "SNLI", "QNLI", "RTE", "WNLI"]
TASK2PATH = {
    "CoLA": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FCoLA.zip?alt=media&token=46d5e637-3411-4188-bc44-5809b5bfb5f4",
    "SST": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FSST-2.zip?alt=media&token=aabc5f6b-e466-44a2-b9b4-cf6337f84ac8",
    "MRPC": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2Fmrpc_dev_ids.tsv?alt=media&token=ec5c0836-31d5-48f4-b431-7480817f1adc",
    "QQP": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FQQP.zip?alt=media&token=700c6acf-160d-4d89-81d1-de4191d02cb5",
    "STS": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FSTS-B.zip?alt=media&token=bddb94a7-8706-4e0d-a694-1109e12273b5",
    "MNLI": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FMNLI.zip?alt=media&token=50329ea1-e339-40e2-809c-10c40afff3ce",
    "SNLI": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FSNLI.zip?alt=media&token=4afcfbb2-ff0c-4b2d-a09a-dbf07926f4df",
    "QNLI": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FQNLIv2.zip?alt=media&token=6fdcf570-0fc5-4631-8456-9505272d1601",
    "RTE": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FRTE.zip?alt=media&token=5efa7e85-a0bb-4f19-8ea2-9e1840f077fb",
    "WNLI": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FWNLI.zip?alt=media&token=068ad0a0-ded7-4bd7-99a5-5e00222e0faf",
}

MRPC_TRAIN = "https://dl.fbaipublicfiles.com/senteval/senteval_data/msr_paraphrase_train.txt"
MRPC_TEST = "https://dl.fbaipublicfiles.com/senteval/senteval_data/msr_paraphrase_test.txt"

data_dir = "data/"
max_seq_length = 128

In [0]:
def download_and_extract(task, data_dir):
    print("Downloading and extracting %s..." % task)
    data_file = "%s.zip" % task
    urllib.request.urlretrieve(TASK2PATH[task], data_file)
    with zipfile.ZipFile(data_file) as zip_ref:
        zip_ref.extractall(data_dir)
    os.remove(data_file)
    print("\tCompleted!")

def format_mrpc(data_dir, path_to_data):
    print("Processing MRPC...")
    mrpc_dir = os.path.join(data_dir, "MRPC")
    if not os.path.isdir(mrpc_dir):
        os.mkdir(mrpc_dir)
    if path_to_data:
        mrpc_train_file = os.path.join(path_to_data, "msr_paraphrase_train.txt")
        mrpc_test_file = os.path.join(path_to_data, "msr_paraphrase_test.txt")
    else:
        print("Local MRPC data not specified, downloading data from %s" % MRPC_TRAIN)
        mrpc_train_file = os.path.join(mrpc_dir, "msr_paraphrase_train.txt")
        mrpc_test_file = os.path.join(mrpc_dir, "msr_paraphrase_test.txt")
        urllib.request.urlretrieve(MRPC_TRAIN, mrpc_train_file)
        urllib.request.urlretrieve(MRPC_TEST, mrpc_test_file)
    assert os.path.isfile(mrpc_train_file), "Train data not found at %s" % mrpc_train_file
    assert os.path.isfile(mrpc_test_file), "Test data not found at %s" % mrpc_test_file
    urllib.request.urlretrieve(TASK2PATH["MRPC"], os.path.join(mrpc_dir, "dev_ids.tsv"))

    dev_ids = []
    with open(os.path.join(mrpc_dir, "dev_ids.tsv"), encoding="utf8") as ids_fh:
        for row in ids_fh:
            dev_ids.append(row.strip().split("\t"))

    with open(mrpc_train_file, encoding="utf8") as data_fh, open(
        os.path.join(mrpc_dir, "train.tsv"), "w", encoding="utf8"
    ) as train_fh, open(os.path.join(mrpc_dir, "dev.tsv"), "w", encoding="utf8") as dev_fh:
        header = data_fh.readline()
        train_fh.write(header)
        dev_fh.write(header)
        for row in data_fh:
            label, id1, id2, s1, s2 = row.strip().split("\t")
            if [id1, id2] in dev_ids:
                dev_fh.write("%s\t%s\t%s\t%s\t%s\n" % (label, id1, id2, s1, s2))
            else:
                train_fh.write("%s\t%s\t%s\t%s\t%s\n" % (label, id1, id2, s1, s2))

    with open(mrpc_test_file, encoding="utf8") as data_fh, open(
        os.path.join(mrpc_dir, "test.tsv"), "w", encoding="utf8"
    ) as test_fh:
        header = data_fh.readline()
        test_fh.write("index\t#1 ID\t#2 ID\t#1 String\t#2 String\n")
        for idx, row in enumerate(data_fh):
            label, id1, id2, s1, s2 = row.strip().split("\t")
            test_fh.write("%d\t%s\t%s\t%s\t%s\n" % (idx, id1, id2, s1, s2))
    print("\tCompleted!")

In [450]:
TASKS = ['SST', 'RTE'] # Или можно просто сюда вписать те датасеты, которые ты выбрал.

for task in TASKS:
    if task == "MRPC":
        format_mrpc(data_dir, None)
    else:
        download_and_extract(task, data_dir)

	Completed!
	Completed!


Загрузи один из выбранных датасет с помощью Pandas(не обязательно через него, но так проще) и посмотри на него.

In [0]:
from sklearn.preprocessing import LabelEncoder

In [0]:
#------------------------------------------------Для датасета RTE
# Вместо test-а возьмите valid, а valid сделай из train.

# <YOUR CODE>
le = LabelEncoder()

train_valid_pd = pd.read_csv('/content/data/RTE/train.tsv', sep='\t').dropna()
train_pd_sen1, valid_pd_sen1 = train_valid_pd['sentence1'][0: int(len(train_valid_pd)*0.7)], train_valid_pd['sentence2'][int(len(train_valid_pd)*0.7) : ]
train_pd_sen2, valid_pd_sen2 = train_valid_pd['sentence2'][0: int(len(train_valid_pd)*0.7)], train_valid_pd['sentence2'][int(len(train_valid_pd)*0.7) : ]
train_pd_labels, valid_pd_labels = train_valid_pd['label'][0: int(len(train_valid_pd)*0.7)], train_valid_pd['label'][int(len(train_valid_pd)*0.7) : ]

# Сбрасываем индексы после разделения
train_pd_sen1.reset_index(drop=True, inplace=True)
train_pd_sen2.reset_index(drop=True, inplace=True)
valid_pd_sen1.reset_index(drop=True, inplace=True)
valid_pd_sen2.reset_index(drop=True, inplace=True)
train_pd_labels.reset_index(drop=True, inplace=True)
valid_pd_labels.reset_index(drop=True, inplace=True)

# Кодируем метки
train_pd_labels = le.fit_transform(train_pd_labels)
valid_pd_labels = le.fit_transform(valid_pd_labels)

test_pd = pd.read_csv('/content/data/RTE/dev.tsv', sep='\t').dropna()
test_pd_sen1, test_pd_sen2, test_pd_labels = test_pd['sentence1'], test_pd['sentence2'], test_pd['label']

# Кодируем метки
test_pd_labels = le.fit_transform(test_pd_labels)
# </YOUR CODE>

In [0]:
# -----------------------------Для датасета SST

# # Вместо test-а возьмите valid, а valid сделай из train.

# # <YOUR CODE>
# train_valid_pd = pd.read_csv('/content/data/SST-2/train.tsv', sep='\t')
# train_pd, valid_pd = train_valid_pd['sentence'][0: int(len(train_valid_pd)*0.7)], train_valid_pd['sentence'][int(len(train_valid_pd)*0.7) : ]
# train_pd_labels, valid_pd_labels = train_valid_pd['label'][0: int(len(train_valid_pd)*0.7)], train_valid_pd['label'][int(len(train_valid_pd)*0.7) : ]
# train_pd.reset_index(drop=True, inplace=True)
# valid_pd.reset_index(drop=True, inplace=True)
# train_pd_labels.reset_index(drop=True, inplace=True)
# valid_pd_labels.reset_index(drop=True, inplace=True)
# test_pd = pd.read_csv('/content/data/SST-2/dev.tsv', sep='\t')
# test_pd, test_pd_labels = test_pd['sentence'], test_pd['label']
# # </YOUR CODE>

# test_pd_labels

Для начала рассмотрим важную часть обработки текста для трансфомера(и не только) – токенайзер.

В качестве примера токенайзера воспользуемся внутренним из библиотеки transformers, обученным для BERT-а. Посмотрим, что он умеет.

In [0]:
model_name = 'bert-base-uncased'

tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

Посмотрим, как происходит токенизация предложения.

In [455]:
test_sentence = "Hide new secretions from the parental units."
print(tokenizer.tokenize(test_sentence))

['hide', 'new', 'secret', '##ions', 'from', 'the', 'parental', 'units', '.']


Видно, что предложения разделяются не на слова, а подслова. Токены, которые надо объеденить в слова для получения "нормального" текста, выделены с помощью `##`. Посмотрим, как различаются коды токенов с этим символом и без него.

In [456]:
print(tokenizer.convert_tokens_to_ids(['ions']))
print(tokenizer.convert_tokens_to_ids(['##ions']))

[15956]
[8496]


Для токенизации предложений воспользуемся методом `encode`. Она принимает предложение как строку или список токенов**(!)**.

In [457]:
print(tokenizer.encode(test_sentence))

[101, 5342, 2047, 3595, 8496, 2013, 1996, 18643, 3197, 1012, 102]


Добавились специальные токены впереди и сзади предложения. Посмотрим на весь список специальных токенов:

In [458]:
print(tokenizer.special_tokens_map)
print({i: j for i, j in zip(tokenizer.all_special_tokens, tokenizer.all_special_ids)})

{'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}
{'[PAD]': 0, '[MASK]': 103, '[UNK]': 100, '[CLS]': 101, '[SEP]': 102}


Посмотрим, что ещё может делать токенайзер. Что требуется нам для обучения BERT-а: добавить паддинг, получить маску аттеншена и тип токенов. Попробуем сделать это самостоятельно и посмотрим, как это сделать с помощью токенайзера.

Выбери два предложения из обучающей выборки. Получи их токены с помощью метода `tokenize`. Объедени списки токенов так, чтобы модель могла различать, что они от разных предложений. 

(Подсказка: на семинаре была картинка с эмбеддингами. Она может подсказать, что надо изменить в токенах предложения) 

In [0]:
# <YOUR CODE>
s1, s2 = train_pd_sen1[0], train_pd_sen1[3]
tokenized_s1, tokenized_s2 = tokenizer.tokenize(s1), tokenizer.tokenize(s2)
s_union = tokenized_s1 + [tokenizer.sep_token] + tokenized_s2
# </YOUR CODE>

# <DON'T TOUCH THIS!>
assert tokenizer.encode(s_union) == tokenizer.encode(s1, s2), "Not equal"
# </DON'T TOUCH THIS!>

Теперь надо добавь нулей в полученный список чисел, чтобы они легко складывались в батчи.

In [0]:
# <YOUR CODE>
u_token = tokenizer.encode(s_union)
encoded_full = u_token + [0] * (max_seq_length - len(u_token))
# </YOUR CODE>

# <DON'T TOUCH THIS!>
encoded_correct = tokenizer.encode(s1, s2, max_length=max_seq_length, pad_to_max_length=True)
assert len(encoded_full) == len(encoded_correct), "Different length"
assert encoded_full == encoded_correct, "Not equal"
# </DON'T TOUCH THIS!>

В модель также надо кинуть маску для механизма внимания и тип предложения для каждого токена. Сделай их.

In [0]:
# <YOUR CODE>
s1_enc_len = len(tokenizer.encode(tokenized_s1))
s2_enc_len = len(tokenizer.encode(tokenized_s2))
s1_enc_len, s2_enc_len
token_type_ids = [0] * s1_enc_len + [1] * (s2_enc_len - 1) + [0] * (len(encoded_full) - s1_enc_len - s2_enc_len + 1)
attention_mask = [1] * (s1_enc_len + s2_enc_len - 1) + [0] * (len(encoded_full) - s1_enc_len - s2_enc_len + 1)
# </YOUR CODE>

# <DON'T TOUCH THIS!>
# encoded_plus = tokenizer.encode_plus(train['sentence'][0], text_pair=train['sentence'][1], max_length=max_seq_length, pad_to_max_length=True)
encoded_plus = tokenizer.encode_plus(s1, s2, max_length=max_seq_length, pad_to_max_length=True)
assert len(token_type_ids) == len(encoded_plus['token_type_ids']), "Different length in token_type_ids"
assert token_type_ids == encoded_plus['token_type_ids'], "Not equal token_type_ids"
assert len(attention_mask) == len(encoded_plus['attention_mask']), "Different length in attention_mask"
assert attention_mask == encoded_plus['attention_mask'], "Not equal attention_mask"
# </DON'T TOUCH THIS!>

Как видно из тестов, все нужные для обработки текста для BERT-а вещи может делать токенизатор из `transformers`. Но не все токенизаторы настолько функциональны. Их (почти)полный список:
- [Sentence Piece](https://github.com/google/sentencepiece/)
- [fastBPE](https://github.com/glample/fastBPE)
- [Hugging Face Tokenizers](https://github.com/huggingface/tokenizers)
- [YouTokenToMe](https://github.com/VKCOM/YouTokenToMe)

Их сравнивают [здесь](https://github.com/VKCOM/YouTokenToMe/blob/master/benchmark.md) или [здесь](https://towardsdatascience.com/a-small-timing-experiment-on-the-new-tokenizers-library-a-write-up-7caab6f80ea6). Также специальные токенайзеры, которые специализируются на "незападные" языки. Но не будем на них останавливаться.

Теперь ты знаешь достаточно, чтобы написать обработчик данных. Что надо сделать: получить из данных предложения, закодировать их, получить аттенш маску и тип токенов, не забыть про таргет. 

P.S. Есть более быстрая версия токенизатора для BERT внутри `transformers`, `BertTokenizerFast`. 

P.S.S. Теперь надо использовать только функционал токенайзера для кодирования предложений, без велосипедов.

In [0]:
class TextClassificationDataset(Dataset):
    def __init__(self, data, labels, tokenizer, data2=None):
        self.data = data
        self.data2 = data2
        self.labels = labels
        self.tokenizer = tokenizer

        # <YOUR CODE>
        self.input_ids = []
        self.attention_mask = []
        self.token_types_ids = []
        self.targets = []

        for i in range(self.data.shape[0]):
          if self.data2 is None:
            encoded_plus = tokenizer.encode_plus(self.data[i], max_length=max_seq_length, pad_to_max_length=True)
          else:
            encoded_plus = tokenizer.encode_plus(self.data[i], self.data2[i], max_length=max_seq_length, pad_to_max_length=True)
          self.input_ids.append(encoded_plus['input_ids'])
          self.attention_mask.append(encoded_plus['attention_mask'])
          self.token_types_ids.append(encoded_plus['token_type_ids'])
          self.targets.append(labels[i])
        # </YOUR CODE>

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.input_ids[idx], dtype=torch.long),
            'attention_mask': torch.tensor(self.attention_mask[idx], dtype=torch.long),
            'token_type_ids': torch.tensor(self.token_types_ids[idx], dtype=torch.long),
            'targets': torch.tensor(self.targets[idx], dtype=torch.long)
        }

Воспользуйтесь семинаром и построй модель для классификации предложений.

(Подсказка: весь код BERT-а из семинара доступен из библиотеки `transformers`)

In [0]:
from transformers import BertConfig
from transformers import BertModel

In [0]:
class BertForSequenceClassification(nn.Module):
    def __init__(self, pretrained_model_name: str, num_labels: int):
        super().__init__()

        # <YOUR CODE>
        config = BertConfig.from_pretrained(pretrained_model_name)
        self.bert = BertModel.from_pretrained(pretrained_model_name)
        self.classifier = nn.Linear(config.hidden_size, num_labels)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        # </YOUR CODE>

    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None):

        # <YOUR CODE>
        bert_output = self.bert(input_ids=input_ids,
                                attention_mask=attention_mask,
                                token_type_ids=token_type_ids)
        hidden_state = bert_output[0]
        pooled_output = hidden_state[:, 0]
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        probas = logits[:, 0]
        # </YOUR CODE>

        return logits, probas

Выбери из [списка](https://huggingface.co/models?search=google%2Fbert_) несколько моделей, которые ты будешь обучать. Сравни их качество на выбранных датасетах. 

Лучше всего будет выбрать одну основную конфигурацию, и другие с небольшим изменением. Например, пройтись по такой сетке: `{'layers': [2, 4], 'num_heads': [2, 4]}`. 

In [465]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# <YOUR CODE>
num_labels = 2
pretrained_model_name = 'google/bert_uncased_L-2_H-128_A-2'
tokenizer = transformers.AutoTokenizer.from_pretrained(pretrained_model_name)
model = BertForSequenceClassification(pretrained_model_name, num_labels=num_labels)
# </YOUR CODE>

model.to(device)
print("Success!")

Success!


In [466]:
%%time

batch_size = 32


# <YOUR CODE>
train_dataset = TextClassificationDataset(data=train_pd_sen1, data2=train_pd_sen2, labels=train_pd_labels, tokenizer=tokenizer)
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=batch_size)

valid_dataset = TextClassificationDataset(data=valid_pd_sen1, data2=valid_pd_sen2, labels=valid_pd_labels, tokenizer=tokenizer)
valid_sampler = RandomSampler(valid_dataset)
valid_dataloader = DataLoader(valid_dataset, sampler=valid_sampler, batch_size=batch_size)

test_dataset = TextClassificationDataset(data=test_pd_sen1, data2=test_pd_sen2, labels=test_pd_labels, tokenizer=tokenizer)
test_sampler = RandomSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=batch_size)

dataloaders = {
    'train': train_dataloader,
    'valid': valid_dataloader,
    'test': test_dataloader
}
# </YOUR CODE>

CPU times: user 2.42 s, sys: 0 ns, total: 2.42 s
Wall time: 2.42 s


In [0]:
seed = 404
set_global_seed(seed)
prepare_cudnn(True)

In [0]:
# Гиперпараметры для обучения модели. Подбери нужные для каждой модели.

epochs = 10
lr = 3e-4
warmup_steps = len(train_dataloader) // 2

Добавь Loss, Optimizer и Scheduler.

In [0]:
optimizer_grouped_parameters = [
    {"params": [p for n, p in model.named_parameters()], "weight_decay": 0.0},
]

# <YOUR CODE>
criterion = torch.nn.CrossEntropyLoss()
optimizer = transformers.AdamW(optimizer_grouped_parameters, lr=lr)
scheduler = transformers.get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=warmup_steps, num_training_steps=epochs
)
# </YOUR CODE>

In [0]:
log_dir = 'logs/'

Для обучения модели воспользуемся библиотекой `catalyst`.

In [471]:
%%time
runner = SupervisedRunner(
    input_key=(
        "input_ids",
        "attention_mask",
        "token_type_ids"
    ),
    output_key = ('logits',
                  'probas'
    ),
    input_target_key = 'targets'
)

runner.train(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    loaders=dataloaders,
    callbacks=[
        AccuracyCallback(num_classes=num_labels,
                         input_key = 'targets',
                         output_key = 'logits'),
        F1ScoreCallback(input_key = 'targets',
                        output_key = 'probas',
                        threshold = 0.5
                        ),
        SchedulerCallback(mode='batch'),
    ],
    logdir=log_dir,
    num_epochs=epochs,
    verbose=False,
)

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning:

To get the last learning rate computed by the scheduler, please use `get_last_lr()`.



[2020-05-06 20:12:24,151] 
1/10 * Epoch 1 (_base): lr=0.000e+00 | momentum=0.9000
1/10 * Epoch 1 (train): accuracy01=0.4748 | f1_score=0.6262 | loss=0.7674 | lr=7.091e-05 | momentum=0.9000
1/10 * Epoch 1 (valid): accuracy01=0.5289 | f1_score=0.6448 | loss=0.6992
1/10 * Epoch 1 (test): accuracy01=0.5428 | f1_score=0.6251 | loss=0.6944


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning:

Please also save or load the state of the optimzer when saving or loading the scheduler.



[2020-05-06 20:12:25,928] 
2/10 * Epoch 2 (_base): lr=0.000e+00 | momentum=0.9000
2/10 * Epoch 2 (train): accuracy01=0.5263 | f1_score=0.6458 | loss=0.7137 | lr=0.000e+00 | momentum=0.9000
2/10 * Epoch 2 (valid): accuracy01=0.5214 | f1_score=0.6546 | loss=0.7010
2/10 * Epoch 2 (test): accuracy01=0.5392 | f1_score=0.6286 | loss=0.6992
[2020-05-06 20:12:27,515] 
3/10 * Epoch 3 (_base): lr=0.000e+00 | momentum=0.9000
3/10 * Epoch 3 (train): accuracy01=0.5193 | f1_score=0.6485 | loss=0.7111 | lr=0.000e+00 | momentum=0.9000
3/10 * Epoch 3 (valid): accuracy01=0.5239 | f1_score=0.6465 | loss=0.7001
3/10 * Epoch 3 (test): accuracy01=0.5355 | f1_score=0.6326 | loss=0.6999
[2020-05-06 20:12:28,993] 
4/10 * Epoch 4 (_base): lr=0.000e+00 | momentum=0.9000
4/10 * Epoch 4 (train): accuracy01=0.5271 | f1_score=0.6431 | loss=0.7094 | lr=0.000e+00 | momentum=0.9000
4/10 * Epoch 4 (valid): accuracy01=0.5289 | f1_score=0.6450 | loss=0.6990
4/10 * Epoch 4 (test): accuracy01=0.5392 | f1_score=0.6342 | loss

Ииии отчет!

Напиши внизу небольшой отчет о проделанной работе. Ожидается сравнение результатов модели с разным количеством голов/слоев на разных датасетах на `test`. Если для оценки качества на датасете используется необычная метрика(не Accuracy или F1), то можно использовать один из них. Было бы круто, если бы вычислялась нужная метрика и она использовалась в отчете.

<ТВОЙ ОТЧЕТ>

</ТВОЙ ОТЧЕТ>

In [0]:
Модель\Задача\Метрика      |    lr    | SST\Acc  ||    lr   | RTE\Acc | RTE\F1  |
___________________________|__________|__________||_________|_________|_________| 
bert_uncased_L-2_H-128_A-2 |  3e-4    |  82.14   ||   3e-4  |  53.74  |  63.72  |
___________________________|__________|__________||_________|_________|_________|
bert_uncased_L-2_H-256_A-4 |  3e-4    |  81.81   ||   7e-5  |  53.54  |  61.88  |
___________________________|__________|__________||_________|_________|_________|
bert_uncased_L-6_H-128_A-2 |  1e-4    |  80.69   ||   1e-4  |  47.44  |  58.99  |
___________________________|__________|__________||_________|_________|_________|
bert_uncased_L-6_H-256_A-4 |  1e-4    |  85.16   ||   1e-4  |  52.15  |  64.00  |
___________________________|__________|__________||_________|_________|_________|

